In [1]:
#@title 1. 의존성 패키지 설치
!pip install -q langchain langchain-community langchain-openai langchain-chroma wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

from google.colab import userdata
import wikipedia

In [3]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [4]:
#@title 2. 데이터 준비
# 한국어 위키피디아로 언어 설정
wikipedia.set_lang("ko")

# 지식 베이스로 사용할 문서 주제 목록
topics = ["인공지능", "머신러닝", "딥러닝"]

# 수집된 내용을 저장할 디렉토리 생성
os.makedirs("ai_wiki_docs", exist_ok=True)

In [5]:
for topic in topics:
    print(f"'{topic}' 주제의 위키피디아 문서를 수집합니다...")
    try:
        # 위키피디아에서 해당 주제의 페이지 객체를 가져옴
        page = wikipedia.page(topic, auto_suggest=False)

        # 페이지의 전체 텍스트 내용을 파일로 저장
        with open(f"ai_wiki_docs/{topic}.txt", "w", encoding="utf-8") as f:
            f.write(page.content)
        print(f"'{topic}.txt' 파일 저장 완료.")
    except wikipedia.exceptions.PageError:
        print(f"'{topic}'에 대한 위키피디아 페이지를 찾을 수 없습니다.")
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"'{topic}'에 대한 여러 페이지가 있어 명확한 지정이 필요합니다: {e.options}")

'인공지능' 주제의 위키피디아 문서를 수집합니다...
'인공지능.txt' 파일 저장 완료.
'머신러닝' 주제의 위키피디아 문서를 수집합니다...
'머신러닝.txt' 파일 저장 완료.
'딥러닝' 주제의 위키피디아 문서를 수집합니다...
'딥러닝.txt' 파일 저장 완료.


In [6]:
!apt install -y tree
!tree -L 2 ai_wiki_docs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (92.1 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
ai_wiki_docs
├── 머신러닝.txt
├── 인공지능.txt
└── 딥러닝.txt

0 directories, 3 files


In [7]:
#@title 3. 임베딩 파이프라인 구성
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- 1. Load: 문서 로드 ---
# 'ai_wiki_docs' 디렉토리의 모든 .txt 파일을 로드한다.
# TextLoader를 사용하여 각 파일의 인코딩을 UTF-8로 지정한다.
loader = DirectoryLoader(
    "./ai_wiki_docs",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
documents = loader.load()

# --- 2. Split: 텍스트 분할 ---
# 문서를 검색에 용이한 작은 조각(Chunk)으로 분할한다.
# chunk_size: 각 조각의 최대 크기 (글자 수 기준)
# chunk_overlap: 조각 간의 중첩되는 글자 수. 문맥 유지를 위해 중요.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# --- 3. Store: 벡터 저장소에 저장 ---
# 분할된 문서를 임베딩하여 ChromaDB에 영구 저장한다.
# persist_directory는 데이터베이스 파일이 저장될 경로를 지정한다.
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY),
    persist_directory="./chroma_db_ai_project"
)

print(f"총 {len(documents)}개의 문서를 {len(splits)}개의 조각으로 분할하여 ChromaDB에 저장했습니다.")

총 3개의 문서를 85개의 조각으로 분할하여 ChromaDB에 저장했습니다.


In [8]:
#@title 4. RAG 체인 구성
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# --- 1. LLM 및 Retriever 준비 ---
llm = ChatOpenAI(
    model_name="gpt-5-mini",
    api_key=OPENAI_API_KEY,
)

# 2단계에서 저장한 ChromaDB를 다시 불러온다.
vectorstore = Chroma(
    persist_directory="./chroma_db_ai_project",
    embedding_function=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
)
retriever = vectorstore.as_retriever()

# --- 2. 질문 재구성(Rephrasing) 체인 ---
# 대화 기록과 새로운 질문을 바탕으로 독립적인 질문을 생성하는 프롬프트
rephrasing_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "주어진 대화 기록을 고려하여, 후속 질문을 검색에 사용할 수 있는 독립적인 질문으로 바꾸어주세요."),
])


# 대화 기록을 고려하여 Retriever를 구성하는 체인.
# 이 체인은 먼저 rephrasing_prompt를 사용하여 질문을 재구성하고, 그 결과로 문서를 검색한다.
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, rephrasing_prompt
)

# --- 3. 문서 검색 및 답변 체인 ---
# 검색된 문서를 바탕으로 최종 답변을 생성하는 프롬프트
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 AI 전문가입니다. 당신의 임무는 오직 주어진 문맥(context) 정보만을 사용하여 사용자의 질문에 답변하는 것입니다.
- 답변의 근거가 된 출처 문서의 이름을 항상 명시해야 합니다. (예: [출처: 인공지능.txt])
- 문맥 정보에 질문에 대한 답변이 포함되어 있지 않다면, 당신의 기존 지식을 활용하지 말고 반드시 "죄송하지만, 제가 가진 정보 내에서는 답변을 찾을 수 없습니다."라고만 답변하세요.
- 절대 외부 정보를 사용하거나 답변을 지어내지 마세요."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("system", "--- 검색된 문맥 ---\n{context}\n--- 문맥 끝 ---"),
])

# 검색된 문서를 프롬프트의 context 변수에 채워넣는 체인
document_chain = create_stuff_documents_chain(llm, answer_prompt)

# --- 4. 두 체인 결합 ---
# history_aware_retriever와 document_chain을 결합하여 최종적인 대화형 RAG 체인을 생성한다.
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever, document_chain
)

print("대화형 RAG 체인 구성 완료.")

대화형 RAG 체인 구성 완료.


In [9]:
#@title 5. 대화 구성 및 실행
from langchain_core.messages import HumanMessage, AIMessage

# 대화 기록을 저장하기 위한 리스트 초기화
chat_history = []

print("안녕하세요! AI 백과사전 챗봇입니다. 무엇이든 물어보세요.")
print("대화를 종료하려면 'exit'을 입력해주세요.")

while True:
    # 사용자로부터 질문을 입력받음
    user_input = input("\n사용자: ")
    if user_input.lower() == 'exit':
        print("챗봇을 종료합니다. 이용해주셔서 감사합니다.")
        break

    # RAG 체인을 호출하여 응답 생성. 대화 기록과 현재 질문을 함께 전달.
    response = conversational_rag_chain.invoke({
        "chat_history": chat_history,
        "input": user_input
    })

    # 챗봇의 답변을 출력
    print("챗봇:", response["answer"])

    # 현재 대화를 기록에 추가
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response["answer"]))

안녕하세요! AI 백과사전 챗봇입니다. 무엇이든 물어보세요.
대화를 종료하려면 'exit'을 입력해주세요.

사용자: 인공지능이 무엇이죠?
챗봇: 인공지능(人工智能, AI)은 인간의 학습능력·추론능력·지각능력을 인공적으로 구현하려는 컴퓨터과학의 한 분야로, 인간(또는 동물)이 가진 자연 지능과는 구별되는 개념입니다. 일반적으로는 인간의 지능을 모방한 기능을 갖춘 컴퓨터 시스템이나, 그런 지능을 만들기 위한 방법론과 연구 분야를 가리키기도 합니다. 실용적 관점에서는 특정 문제를 해결하는 약인공지능(weak AI)과, 인간처럼 범용적 사고로 문제를 해결하려는 강인공지능(AGI, 강한 인공지능)으로 구분됩니다.  
[출처: 인공지능.txt]

사용자: 그건 누가 주도했나요?
챗봇: 인공지능은 특정 개인 한 명이 아니라 여러 연구자들과 기관들이 주도했습니다. 문맥에 언급된 주요 인물과 기관은 다음과 같습니다.

- 마빈 민스키  
- 볼프강 발스터(Wolfgang Wahlster)  
- 존 매카시  
- 더글러스 레넛(Doug Lenat)  
- 로저 섕크  
- 앨런 튜링  
- 라지 레디(Raj Reddy)  
- 테리 위노그래드(Terry Winograd)  
- 로드니 브룩스(Rodney Brooks)  
- 스튜어트 러셀(Stuart Russell)  
- 페퍼트 등  

또한 MIT 인공지능 연구소와 같이 특정 연구소들이 인공지능 연구의 모태가 되었다고 문맥에서 언급하고 있습니다.  
[출처: 인공지능.txt]

사용자: 알파고를 개발한 회사는 어디인가요?
챗봇: 죄송하지만, 제가 가진 정보 내에서는 답변을 찾을 수 없습니다. [출처: 인공지능.txt]

사용자: exit
챗봇을 종료합니다. 이용해주셔서 감사합니다.
